In [111]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [112]:
import unidecode
import string
import random
import re

In [113]:
all_character = string.printable #All characters 
n_character = len(all_character)

with open("data/shakespeare.txt", "r") as f:
    file = unidecode.unidecode(f.read())
    file_len = len(file)
print(file_len)

1115394


In [114]:
def random_chunk(chunk_len = 200):
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len +1 
    return file[start_index:end_index]

In [115]:
def char2tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_character.index(string[c])
    return Variable(tensor)

In [122]:
def random_training_set(chunk_len=200, batch_size=100):
    
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len)
        end_index = start_index + chunk_len + 1
        chunk = file[start_index:end_index]
        inp[bi] = char2tensor(chunk[:-1])
        target[bi] = char2tensor(chunk[1:])
    inp = Variable(inp)
    target = Variable(target)
    return inp, target

In [133]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        batch_size = input.size(0)
        input = self.encoder(input)
        print(input.shape)
        output, hidden = self.gru(input.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))


In [134]:
input_size = n_character #
hidden_size = 100
output_size = n_character

model = RNN(input_size,hidden_size, output_size)

In [135]:
inp, target = random_training_set()
print(inp.shape)
hidden = model.init_hidden(100)
outputs,hidden = model(inp,hidden)

torch.Size([100, 200])
torch.Size([100, 200, 100])


RuntimeError: input.size(-1) must be equal to input_size. Expected 100, got 20000

In [85]:
decoder = RNN(n_character,100,n_character,1)
print(decoder)

RNN(
  (encoder): Embedding(100, 100)
  (gru): GRU(100, 100)
  (decoder): Linear(in_features=100, out_features=100, bias=True)
)


In [86]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted

In [87]:
def train_step(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c])

    loss.backward()
    decoder_optimizer.step()

    return loss.data[0] / chunk_len

In [88]:
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [89]:
def train(decoder):
    EPOCHS = 200
    
    
    start = time.time()
    total_losses = []
    loss_avg = 0
    
    for epoch in range(EPOCHS):
        loss = train_step(*random_training_set())  
        total_losses.append(loss)
        if epoch % 100 == 0:
            print("Time: {}, Epoch: [{}/{}], Loss: {}".format(time_since(start),epoch,EPOCHS,loss))
            with torch.no_grad():
                print(evaluate(decoder,'Wh',100))
    torch.save(decoder.state_dict(),"model/model.pth")
    return decoder, total_losses

In [90]:
TRAIN_DATA = 1
if TRAIN_DATA:
    optimizer = torch.optim.Adam(decoder.parameters(), lr=0.0002)
    criterion = nn.CrossEntropyLoss()
    decoder, total_loss = train(decoder)

IndexError: dimension specified as 0 but tensor has no dimensions